In [1]:
import pandas as pd
from keras.callbacks import History, ReduceLROnPlateau,EarlyStopping,ModelCheckpoint
import os
import numpy as np
from data_analysis import calculate_metrics, load_weights_and_evaluate
from model_builders import GCN_pretraining
from hyperparameter_tuning_GCN import objective
from functools import partial
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
import pickle
import dill
from hyper_mining import objective_fn
import keras.backend as K
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.python.ops import array_ops
from tensorflow.python.ops import math_ops
from tensorflow.python.framework import dtypes
import keras
import keras.backend as K
from keras.layers import Dense, Dropout, Input, Lambda, concatenate,Flatten
from keras.models import Model, load_model
import seaborn as sns
from keras.callbacks import History, ReduceLROnPlateau,EarlyStopping,ModelCheckpoint
from distance_and_mask_fn import pairwise_distance,masked_maximum,masked_minimum

from NGF.preprocessing import tensorise_smiles
from custom_layers.model_creator import encode_smiles, stage_creator
from data_analysis import calculate_metrics
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, roc_curve, roc_auc_score, auc, average_precision_score
from hyperopt import STATUS_OK
import xgboost as xgb

Using TensorFlow backend.
C:\Users\tomas\miniconda3\envs\binding\lib\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\tomas\miniconda3\envs\binding\lib\site-packages\tensorflow\python\framework\dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\tomas\miniconda3\envs\binding\lib\site-packages\tensorflow\python\framework\dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\tomas\miniconda3\envs\binding

In [2]:
fspace = {
    'conv1' : hp.quniform('conv1', 32, 64, 8),
    'conv2' : hp.quniform('conv2', 64, 128, 8),
    'conv3' : hp.quniform('conv3', 128, 168, 8),
    'fp' : hp.quniform('fp', 96, 196, 8),
    'dense1' : hp.quniform('dense1',96,512,32),
    'dense2' : hp.quniform('dense2',96,512,32),
    'dense3' : hp.quniform('dense3',64,512,32),
    'dropout_rate' : hp.uniform('dropout_rate',0.1,0.5),
    'lr' : hp.uniform('lr',0.00001,0.01),
    'n_epochs' : hp.quniform('n_epochs',15,60,5),
    'batch_size' : hp.quniform('batch_size',64,256,16),
    'colsample_bylevel' : hp.uniform('colsample_bylevel', 0.1, 1), 
    'colsample_bytree' : hp.uniform('colsample_bytree', 0.1, 1), 
    'gamma' : hp.uniform('gamma', 0.1, 1), 
    'learning_rate' : hp.uniform('learning_rate', 0.1, 1),
    'max_delta_step' : hp.quniform('max_delta_step',1,10,1),
    'max_depth' : hp.quniform('max_depth',6, 12, 1),
    'min_child_weight' : hp.quniform('min_child_weight',10 ,500 ,5),
    'reg_alpha' : hp.uniform('reg_alpha',0.1,100),
    'reg_lambda' : hp.uniform('reg_lambda',0.1,100),
    'subsample' : hp.uniform('subsample',0.1,1.0),
    'max_bin' : hp.quniform('max_bin',16,256,16),
    'margin' : hp.uniform('margin',0.2,2)
}

In [3]:
target = 'p38'
base_path = f'C:/Users/tomas/Documents/GitHub/kinase_binding'

data_fpath = base_path+f'/data/{target}/data.csv'
df=pd.read_csv(data_fpath).set_index('biolab_index')

with open(base_path+f'/data/{target}/train_val_folds.pkl', "rb") as in_f:
    train_val_folds = dill.load(in_f)
with open(base_path+f'/data/{target}/train_test_folds.pkl', "rb") as in_f:
    train_test_folds = dill.load(in_f)

In [4]:
training_list = [df.loc[train_val_folds[0][0]],
                 df.loc[train_val_folds[1][0]],
                 df.loc[train_val_folds[2][0]],
                 df.loc[train_val_folds[3][0]],
                 df.loc[train_val_folds[4][0]],
                 df.loc[train_val_folds[5][0]],
                 ]
validation_list = [df.loc[train_val_folds[0][1]],
                   df.loc[train_val_folds[1][1]],
                   df.loc[train_val_folds[2][1]],
                   df.loc[train_val_folds[3][1]],
                   df.loc[train_val_folds[4][1]],
                   df.loc[train_val_folds[5][1]],
                   ]

In [5]:
fmin_objective = partial(objective, train_sets = training_list, val_sets = validation_list)

In [6]:
def run_trials():

    trials_step = 0  # how many additional trials to do after loading saved trials. 1 = save after iteration
    max_trials = 1  # initial max_trials. put something small to not have to wait

    
    try:  # try to load an already saved trials object, and increase the max
        trials = pickle.load(open("gcn_xgb.hyperopt", "rb"))
        print("Found saved Trials! Loading...")
        max_trials = len(trials.trials) + trials_step
        print("Rerunning from {} trials to {} (+{}) trials".format(len(trials.trials), max_trials, trials_step))
    except:  # create a new trials object and start searching
        trials = Trials()

    best = fmin(fn = fmin_objective, space = fspace, algo=tpe.suggest, max_evals=max_trials, trials=trials)

    print("Best:", best)
    
    # save the trials object
    with open("gcn_xgb.hyperopt", "wb") as f:
        pickle.dump(trials, f)
    return(trials)

In [7]:
trials = run_trials()

Found saved Trials! Loading...
Rerunning from 73 trials to 73 (+0) trials
100%|██████████████████████████████████████████████████████████████████████████| 73/73 [00:00<?, ?trial/s, best loss=?]
Best: {'batch_size': 96.0, 'colsample_bylevel': 0.19610957495328543, 'colsample_bytree': 0.9504328963958085, 'conv1': 88.0, 'conv2': 96.0, 'conv3': 160.0, 'dense1': 96.0, 'dense2': 416.0, 'dense3': 320.0, 'dropout_rate': 0.45547284530546817, 'fp': 160.0, 'gamma': 0.6648409346047757, 'learning_rate': 0.9073217403165388, 'lr': 0.0019499018534396453, 'margin': 0.6011401246738063, 'max_bin': 208.0, 'max_delta_step': 1.0, 'max_depth': 8.0, 'min_child_weight': 355.0, 'n_epochs': 60.0, 'reg_alpha': 56.190739151936484, 'reg_lambda': 52.450354051682496, 'subsample': 0.8316785844601918}


In [ ]:
from numba import cuda 
device = cuda.get_current_device()
device.reset()

In [8]:
best_loss = trials.trials[0]['result']['loss']
for i in range(1,len(trials.trials)):
    if (trials.trials[i]['result']['loss'] <=  best_loss):
        best_loss = trials.trials[i]['result']['loss']
        index = i
best_params = trials.trials[index]['misc']['vals']

In [21]:
trials.trials[index]

{'state': 2,
 'tid': 28,
 'spec': None,
 'result': {'loss': -0.887696524475951, 'status': 'ok'},
 'misc': {'tid': 28,
  'cmd': ('domain_attachment', 'FMinIter_Domain'),
  'workdir': None,
  'idxs': {'batch_size': [28],
   'colsample_bylevel': [28],
   'colsample_bytree': [28],
   'conv1': [28],
   'conv2': [28],
   'conv3': [28],
   'dense1': [28],
   'dense2': [28],
   'dense3': [28],
   'dropout_rate': [28],
   'fp': [28],
   'gamma': [28],
   'learning_rate': [28],
   'lr': [28],
   'margin': [28],
   'max_bin': [28],
   'max_delta_step': [28],
   'max_depth': [28],
   'min_child_weight': [28],
   'n_epochs': [28],
   'reg_alpha': [28],
   'reg_lambda': [28],
   'subsample': [28]},
  'vals': {'batch_size': [96.0],
   'colsample_bylevel': [0.19610957495328543],
   'colsample_bytree': [0.9504328963958085],
   'conv1': [88.0],
   'conv2': [96.0],
   'conv3': [160.0],
   'dense1': [96.0],
   'dense2': [416.0],
   'dense3': [320.0],
   'dropout_rate': [0.45547284530546817],
   'fp': [160

In [9]:
from hyper_mining import XGB_predictor,GCN_online_mining_test
from data_analysis import calculate_metrics
es = EarlyStopping(monitor='loss',patience=8, min_delta=0)
rlr = ReduceLROnPlateau(monitor='loss',factor=0.5, patience=4, verbose=1, min_lr=0.0000001)
gcn_best = {
        "num_layers" : 3,
        "max_atoms" : 70,
        "num_atom_features" : 62,
        "num_atom_features_original" : 62,
        "num_bond_features" : 6,
        "max_degree" : 5,
        "conv_width" : [int(best_params['conv1'][0]), int(best_params['conv2'][0]), int(best_params['conv3'][0])],
        "fp_length" : [int(best_params['fp'][0]), int(best_params['fp'][0]), int(best_params['fp'][0])],
        "activ_enc" : "selu",
        "activ_dec" : "selu",
        "learning_rates" : [0.001,0.001,0.001],
        "learning_rates_fp": [0.005,0.005,0.005],
        "losses_conv" : {
                    "neighbor_output": "mean_squared_error",
                    "self_output": "mean_squared_error",
                    },
        "lossWeights" : {"neighbor_output": 1.0, "self_output": 1.0},
        "metrics" : "mse",
        "loss_fp" : "mean_squared_error",
        "enc_layer_names" : ["enc_1", "enc_2", "enc_3"],
        'callbacks' : [es,rlr],
        'adam_decay': 0.0005329142291371636,
        'beta': 5,
        'p': 0.004465204118126482,
        'dense_size' : [int(best_params['dense1'][0]), int(best_params['dense2'][0]), int(best_params['dense3'][0])],
        'dropout_rate' : [best_params['dropout_rate'][0], best_params['dropout_rate'][0]],
        'lr' : best_params['lr'][0],
        'batch_size' : int(best_params['batch_size'][0]),
        'n_epochs' : int(best_params['n_epochs'][0]),
        'margin' : best_params['margin'][0]
        }
xgb_best = {
        "colsample_bylevel" : best_params['colsample_bylevel'][0],
        "colsample_bytree" : best_params['colsample_bytree'][0],
        "gamma" : best_params['gamma'][0],
        "eta" : best_params['learning_rate'][0],
        "max_delta_step" : int(best_params['max_delta_step'][0]),
        "max_depth" : int(best_params['max_depth'][0]),
        "min_child_weight" : int(best_params['min_child_weight'][0]),
        "alpha" : best_params['reg_alpha'][0],
        "lambda" : best_params['reg_lambda'][0],
        "subsample" : best_params['subsample'][0],
        "max_bin" : int(best_params['max_bin'][0]),
        "eval_metric":'auc',
        "objective":'binary:logistic',
        "booster":'gbtree'
        #"single_precision_histogram" : True
        }
class_XGB = XGB_predictor(xgb_best)
class_GCN = GCN_online_mining_test(gcn_best)

In [13]:
xgb_hyper = {
        "colsample_bylevel" : 0.5612301667238877,
        "colsample_bytree" : 0.788688363076523,
        "gamma" : 0.35376030016117566,
        "eta" : 0.4023692255888918,
        "max_delta_step" : int(3),
        "max_depth" : int(8),
        "min_child_weight" : int(70),
        "alpha" : 0.15030685758880047,
        "lambda" : 15.311721955443915,
        "subsample" : 0.8303923929525608,
        "max_bin" : int(208),
        "eval_metric":'auc',
        "objective":'binary:logistic',
        "booster":'gbtree',
        "single_precision_histogram" : True
}
class_XGB_2 = XGB_predictor(xgb_hyper)

In [14]:
K.clear_session()
training_metrics = {}
validation_metrics = {}
es2 = EarlyStopping(monitor='loss',patience=15, min_delta=0)
rlr2 = ReduceLROnPlateau(monitor='loss',factor=0.5, patience=2, verbose=1, min_lr=0.000000001)
for i in range(0,1):#range(len(training_list)):
        X_atoms_cold,X_bonds_cold,X_edges_cold = class_GCN.dataframe_to_gcn_input(validation_list[i])
        Y_cold = validation_list[i].Binary 
        Y_dummy_cold = np.empty((X_atoms_cold.shape[0],gcn_params['dense_size'][2]+1))
        X_atoms_train, X_bonds_train, X_edges_train = class_GCN.dataframe_to_gcn_input(training_list[i])
        Y = training_list[i].Binary
        Y_dummy_train = np.empty((X_atoms_train.shape[0],gcn_params['dense_size'][2]+1))
        
        gcn_encoder = class_GCN.build_encoder()
        gcn_model = class_GCN.build_model(gcn_encoder)
        gcn_mining = class_GCN.build_mining(gcn_model)
        
        gcn_mining.fit([X_atoms_train,X_bonds_train,X_edges_train,Y],
                       Y_dummy_train,
                       epochs = gcn_params['n_epochs'],
                       batch_size = gcn_params['batch_size'],
                       shuffle = True,
                       validation_data = ([X_atoms_cold,X_bonds_cold,X_edges_cold,Y_cold],Y_dummy_cold),
                       callbacks=[es2,rlr2]
                      )
        #Predict Embeddings
        embeddings_cold = gcn_model.predict([X_atoms_cold,X_bonds_cold,X_edges_cold])
        embeddings_train = gcn_model.predict([X_atoms_train, X_bonds_train, X_edges_train])
        
        #Prepare data for XGBoost
        dmatrix_train = class_XGB_2.to_xgb_input(Y,embeddings_train)
        dmatrix_cold = class_XGB_2.to_xgb_input(Y_cold,embeddings_cold)
        
        evalist = [(dmatrix_train,'train'),(dmatrix_cold,'eval')]
        xgb_model = class_XGB_2.build_model(dmatrix_train,evalist,300)
        
        xgb_pred_cold = xgb_model.predict(dmatrix_cold)
        validation_metrics['Val_%s'%i] = calculate_metrics(np.array(Y_cold),xgb_pred_cold)
        
        xgb_pred_train = xgb_model.predict(dmatrix_train)
        training_metrics['Train_%s'%i] = calculate_metrics(np.array(Y),xgb_pred_train)
        


LAYER 0
LAYER 1
LAYER 2


C:\Users\tomas\miniconda3\envs\binding\lib\site-packages\tensorflow\python\ops\gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 2541 samples, validate on 509 samples
Epoch 1/60
2541/2541 [==============================] - 5s 2ms/step - loss: 0.5916 - val_loss: 0.5880
Epoch 2/60
2541/2541 [==============================] - 1s 395us/step - loss: 0.5805 - val_loss: 0.5770
Epoch 3/60
2541/2541 [==============================] - 1s 367us/step - loss: 0.5702 - val_loss: 0.5393
Epoch 4/60
2541/2541 [==============================] - 1s 380us/step - loss: 0.5606 - val_loss: 0.5404
Epoch 5/60
2541/2541 [==============================] - 1s 365us/step - loss: 0.5604 - val_loss: 0.5467
Epoch 6/60
2541/2541 [==============================] - 1s 379us/step - loss: 0.5580 - val_loss: 0.5460
Epoch 7/60
2541/2541 [==============================] - 1s 365us/step - loss: 0.5543 - val_loss: 0.5229
Epoch 8/60
2541/2541 [==============================] - 1s 374us/step - loss: 0.5582 - val_loss: 0.5319
Epoch 9/60
2541/2541 [==============================] - 1s 364us/step - loss: 0.5499 - val_loss: 0.5038
Epoch 10/60
2541/25

In [15]:
validation_metrics

{'Val_0': {'roc_auc': 0.8363488119273179,
  'tn': 199,
  'fp': 75,
  'fn': 36,
  'tp': 199,
  'map': 0.7909681465148516,
  'precision': 0.7262773722627737,
  'recall': 0.8468085106382979,
  'accuracy': 0.7819253438113949}}

In [16]:
training_metrics

{'Train_0': {'roc_auc': 0.9576354582117707,
  'tn': 1081,
  'fp': 211,
  'fn': 85,
  'tp': 1164,
  'map': 0.9566848361539488,
  'precision': 0.8465454545454546,
  'recall': 0.9319455564451561,
  'accuracy': 0.8835104289649744}}

In [13]:
validation_metrics

{'Val_0': {'roc_auc': 0.5,
  'tn': 274,
  'fp': 0,
  'fn': 235,
  'tp': 0,
  'map': 0.46168958742632615,
  'precision': 0.0,
  'recall': 0.0,
  'accuracy': 0.5383104125736738},
 'Val_1': {'roc_auc': 0.5,
  'tn': 295,
  'fp': 0,
  'fn': 214,
  'tp': 0,
  'map': 0.4204322200392927,
  'precision': 0.0,
  'recall': 0.0,
  'accuracy': 0.5795677799607073},
 'Val_2': {'roc_auc': 0.5,
  'tn': 263,
  'fp': 0,
  'fn': 246,
  'tp': 0,
  'map': 0.48330058939096265,
  'precision': 0.0,
  'recall': 0.0,
  'accuracy': 0.5166994106090373},
 'Val_3': {'roc_auc': 0.5,
  'tn': 262,
  'fp': 0,
  'fn': 247,
  'tp': 0,
  'map': 0.48526522593320237,
  'precision': 0.0,
  'recall': 0.0,
  'accuracy': 0.5147347740667977},
 'Val_4': {'roc_auc': 0.5,
  'tn': 228,
  'fp': 0,
  'fn': 281,
  'tp': 0,
  'map': 0.5520628683693517,
  'precision': 0.0,
  'recall': 0.0,
  'accuracy': 0.44793713163064836},
 'Val_5': {'roc_auc': 0.5,
  'tn': 244,
  'fp': 0,
  'fn': 261,
  'tp': 0,
  'map': 0.5168316831683168,
  'precision

In [14]:
training_metrics

{'Train_0': {'roc_auc': 0.5,
  'tn': 1292,
  'fp': 0,
  'fn': 1249,
  'tp': 0,
  'map': 0.491538764266037,
  'precision': 0.0,
  'recall': 0.0,
  'accuracy': 0.508461235733963},
 'Train_1': {'roc_auc': 0.5,
  'tn': 1271,
  'fp': 0,
  'fn': 1270,
  'tp': 0,
  'map': 0.49980322707595437,
  'precision': 0.0,
  'recall': 0.0,
  'accuracy': 0.5001967729240456},
 'Train_2': {'roc_auc': 0.5,
  'tn': 1303,
  'fp': 0,
  'fn': 1238,
  'tp': 0,
  'map': 0.48720975993703264,
  'precision': 0.0,
  'recall': 0.0,
  'accuracy': 0.5127902400629674},
 'Train_3': {'roc_auc': 0.5,
  'tn': 1304,
  'fp': 0,
  'fn': 1237,
  'tp': 0,
  'map': 0.4868162140889414,
  'precision': 0.0,
  'recall': 0.0,
  'accuracy': 0.5131837859110586},
 'Train_4': {'roc_auc': 0.5,
  'tn': 1338,
  'fp': 0,
  'fn': 1203,
  'tp': 0,
  'map': 0.4734356552538371,
  'precision': 0.0,
  'recall': 0.0,
  'accuracy': 0.526564344746163},
 'Train_5': {'roc_auc': 0.5,
  'tn': 1322,
  'fp': 0,
  'fn': 1223,
  'tp': 0,
  'map': 0.48055009823